# Equations and theoretical model

### Theoretical model
<div>
<img src="img/pendulum_coords.png"/>
<img src="img/polar_coords.png"/>
</div>
<br>

The cartesian coordinates o-xyz represent the pendulum's coordinate system. In order to describe its position we use the angles $\alpha$ and $\beta$:

> x = L $\sin\alpha \cos\beta$

> y = L $sin\alpha \sin\beta$

> z = $h_{0} - L \cos\alpha$

Where: L represents the pendulum's length and $h_{0}$ the distance from the center of the Earth to the pendulum's bob
<br>

Besides the pendulum's oscillating motion, it also rotates with the earth. In order to describe the angular velocity we require an assistant coordinate system O-XYZ. Here, the pendulum sits on a fixed point described using the spherical coordinates $\phi$ (latitude) and $\theta$ (longitude).

### Equations

In order to compute the pendulum's coordinates, we use the following two coupled second order differential equations to find $\alpha$ and $\beta$:

> $\ddot{\alpha}  -  \sin\alpha \cos\alpha \dot{\beta}^2  -  2 \omega \sin\alpha (\cos\phi \sin\alpha \cos\beta + \sin\phi \cos\alpha) \dot{\beta}  +  \frac{g}{l} \sin\alpha$ = 0

> $\sin\alpha \ddot{\beta}  +  \cos\alpha \dot{\alpha} \dot{\beta}  +  2 \omega (\cos\phi \sin\alpha \cos\beta + \sin\phi \cos\alpha) \dot{\alpha}$ = 0
 
Where: $\omega$ is the Earth's angular velocity and <i>g</> is the Earth's gravitational acceleration
<br>

To solve the ODEs we utilize the function <code>odeint()</code> from <code>scipy.integrate</code>:

In [ ]:
from scipy.integrate import odeint

def solve_ode(ode, initial_conditions, time_values, index):
    solution = odeint(func = ode, y0 = initial_conditions, t = time_values)
    return solution.T[index]

The first parameter required is a callable function that contains our ODEs. The caveat of <code>odeint</code> is that it can only solve first order differential equations. As such, we have to rewrite our equations so that it only contains first order ODEs.

We start by defining two new variables, <i>x</i> and <i>y</i>, as such:
(Note that x and y in this situation are not related to the pendulum's coordinates x and y)

> x = $\dot{\alpha}  \to  \dot{x} = \ddot{\alpha}$

> y = $\dot{\beta}  \to  \dot{y} = \ddot{\beta}$

<br>
And now we can rewrite our initial equations using x and y:

> $\dot{x} = \sin\alpha \cos\alpha y^2  +  2 \omega \sin\alpha (\cos\phi \sin\alpha \cos\beta + \sin\phi \cos\alpha) y  -  \frac{g}{l} \sin\alpha$

> $\dot{y} = ( - 2 \cos\alpha x y  -  2 \omega (\cos\phi \sin\alpha \cos\beta + \sin\phi \cos\alpha) x ) \sin\alpha$ 

<br>
And now using these four equations we can define our callable function <code>ode()</code>: 

In [ ]:
import numpy as np

def ode(initial_conditions, t):
    a, b, x, y = initial_conditions
    da, db = x, y
    dx = (np.sin(a) * np.cos(a) * y**2 + 2 * omega * np.sin(a) * (np.cos(phi) * np.sin(a) * np.cos(b) + np.sin(phi) * np.cos(a)) * y
         - (g/L) * np.sin(a))
    dy = ((-2 * np.cos(a) * x * y - 2 * omega * (np.cos(phi) * np.sin(a) * np.cos(b) + np.sin(phi) * np.cos(a)) * x)
         / np.sin(a))
    return [da, db, dx, dy]

Where a, b, x, y are $\alpha$, $\beta$, <i>x</i>, <i>y</i> and da, db, dx, dy are $\dot{\alpha}$, $\dot{\beta}$, $\dot{x}$, $\dot{y}$. 
<br>

The function <code>ode</code> gets called by <code?odeint</code> with the parameters: <i>initial_conditions</i> which is an array that contains the initial conditions on y, and t, an array that contains a sequance of monotonically increasing time points for which to solve for y.

Besides the initial condition, the equations also require the constants: phi, omega, g and L. The default values are:

In [ ]:
# Constants
L = 30 #Length of pendulum in meters
g = 9.81 #Gravitational acceleration, m/s^2
omega = 7.27e-5 #Angular velocity of Earth, rad/s
phi = np.radians(45) #φ,Latitude of pendulum

# Initial conditions
a_0 = np.radians(10)
b_0 = np.radians(0)
x_0 = 0.1
y_0 = 0.1
initial_conditions = [a_0, b_0, x_0, y_0]

# Time interval
start_time = 0
end_time = 1000
time_values = np.arange(start = start_time, stop = end_time, step = 0.01)